# Oral Cancer Classification

## Train Test Split

In [25]:
import os
import shutil
import pathlib

original_dir = pathlib.Path("Content_3class_new")
new_base_dir = pathlib.Path("oral_cancer_train_test_split")

def make_subset(subset_name, categories):
    for category, (start_index, end_index) in categories.items():
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)

        for i in range(start_index, end_index):
            for ext in ['.jpg', '.jpeg', '.png']:
                fname = f"{category}.{i}{ext}"
                src = original_dir / category/ fname
                if src.exists(): 
                    shutil.copy(src, dir / fname)
                    break

# images in each category
categories = {
    "normal": (0, 213),      # 80% of 266 for training
    "pre_cancer": (0, 249),   # 80% of 312 for training
    "oral_cancer": (0, 226),      # 80% of 283 for training
}

# training set
make_subset("train", categories)

# categories for testing
categories = {
    "normal": (213, 266),    # Remaining 20% for testing
    "pre_cancer": (249, 312),  # Remaining 20% for testing
    "oral_cancer": (226, 283),     # Remaining 20% for testing
}

# testing set
make_subset("test", categories)

print('Created training and testing subsets')

Created training and testing subsets
